# Лабораторная работа №3
# Решение задач линейного программирования при помощи двойственного симплекс-метода
---
Выполнил: Совпель Дмитрий, гр. 853501

## Реализация метода

Импортируем необходимые библиотеки.

In [1]:
import typing as tp
import warnings

import numpy as np
import scipy.linalg as sla
import scipy.optimize as opt

Реализуем функцию обращения матрицы также, как и в предыдущей лабораторной работе.

In [2]:
def inverse(A: np.array) -> np.array:
    return sla.solve(A, np.eye(A.shape[0]))

Реализуем двойственный симплекс метод, а также метод `get_solution` для получения решения задачи по базисным компонентам плана и самому базису.

In [24]:
def dual_simplex_method(
    A: np.array,
    b: np.array,
    c: np.array,
    initial_solution: np.array,
    initial_basis: np.array,
    epsilon: float = 1e-7,
    verbose: bool = False,
) -> tp.Tuple[np.array, np.array]:
    solution = initial_solution.copy()
    cosolution = solution @ A - c
    basis = initial_basis.copy()
    non_basis = np.array(list(set(range(c.size)) - set(basis)), dtype="int")

    A_basis = A[:, basis]
    inverse_A_basis = inverse(A_basis)

    iteration = 0
    while True:
        if verbose:
            print(f"Iteration no.: {iteration}")
            print(f"Solution: {solution}")
            print(f"Basis: {basis + 1}")
            print(f"Cosolution: {cosolution}")
            print(f"A_B: \n{A_basis}")
            print(f"Inversed A_B: \n{inverse_A_basis}")

        chi = inverse_A_basis @ b
        if verbose:
            print(f"x_B: {chi}")

        k = np.argmax(chi < -epsilon)
        j_k = basis[k]
        if chi[k] < -epsilon:
            if verbose:
                print(f'k: {k + 1}')
                print(f'j_k: {j_k + 1}')
            
            mu_j = inverse_A_basis[k] @ A
            negative_mu_j_indexes = np.ravel(np.argwhere(mu_j[non_basis] < -epsilon))

            if verbose:
                print(f"mu_j: {mu_j}")

            if negative_mu_j_indexes.size == 0:
                warnings.warn("No solution", opt.OptimizeWarning)
                return None
            sigma = (
                -cosolution[non_basis][negative_mu_j_indexes]
                / mu_j[non_basis][negative_mu_j_indexes]
            )
            if verbose:
                print(f"Sigma: {sigma}")

            minimum_sigma_index = np.argmin(sigma)
            minimum_sigma = sigma[minimum_sigma_index]

            s = negative_mu_j_indexes[minimum_sigma_index]
            j_s = non_basis[s]
            
            if verbose:
                print(f's: {s + 1}')
                print(f'j_s: {j_s + 1}')
            
            solution += minimum_sigma * inverse_A_basis[k]
            cosolution = solution @ A - c

            basis[k] = j_s
            non_basis[s] = j_k
            A_basis = A[:, basis]
            alpha = inverse_A_basis[k] @ A[:, j_s]
            inverse_A_basis -= (
                (inverse_A_basis @ (A[:, j_s] - A[:, j_k])).reshape(-1, 1)
                @ (inverse_A_basis[k]).reshape(1, -1)
                / alpha
            )
        else:
            return chi, basis
        iteration += 1


def get_solution(solution: np.array, basis: np.array, n: int) -> np.array:
    result = np.zeros(n)
    result[basis] = solution
    return result

## Тестирование реализованного алгоритма на линейной программе с несовместными ограничениями

Протестируем реализованный алгоритм на линейной программе, заданной в канонической форме: 
$$
    \begin{cases}
      c^Tx \rightarrow max \\
      Ax = b \\
      x > 0 \\
     \end{cases}
$$
где: 
$$
    \begin{aligned} 
        A &= \begin{pmatrix}
        2 & -3 & 4 & 0 & 2 & 0 & -2 & 7 \\
        0 & 2 & -2 & 2 & 1 & -1 & 4 & 3 \\ 
        0 & 4 & 1 & 1 & 3 & 5 & 1 & 2
    \end{pmatrix} \\ \\
    b &= \begin{pmatrix} 0 & -4 & -18\end{pmatrix} \\ \\
    c &= \begin{pmatrix}-2 & 15 & -11 & 8 & 3 & -2 & 13 & 7\end{pmatrix} 
    \end{aligned}
$$
при этом необходимо взять начальный двойственный план $y^0$:
$$
    y^0 = \begin{pmatrix} -1 & 4 & 1\end{pmatrix}
$$
и базис:
$$
    J = \{1, 2, 3\}
$$

Как мы видим, данный алгоритм выдает предупреждение о том, что для данной линейной программы нет решения

In [160]:
A = np.array(
    [[2, -3, 4, 0, 2, 0, -2, 7], [0, 2, -2, 2, 1, -1, 4, 3], [0, 4, 1, 1, 3, 5, 1, 2]],
    dtype="float32",
)

b = np.array([0, -4, -18], dtype="float32")
c = np.array([-2, 15, -11, 8, 3, -2, 13, 7], dtype="float32")

initial_solution = np.array([-1, 4, 1], dtype="float32")
initial_basis = np.array([0, 1, 2])

dual_simplex_method(A, b, c, initial_solution, initial_basis)

<ipython-input-157-6af4ff6b819d>:42: OptimizeWarning: No solution
  warnings.warn("No solution", opt.OptimizeWarning)


Также, можно проверить результат при помощи симплекс-метода реализованного в функции `linprog` из библиотеки `scipy`. Как видно, функция также выдает сообщение, о том что у данной линейной программы нет решения.

In [118]:
opt.linprog(A_eq=A, b_eq=b, c=-c, method="revised simplex")

     con: array([  0.,  -4., -18.])
     fun: 0.0
 message: 'The problem appears infeasible, as the phase one auxiliary problem terminated successfully with a residual of 2.2e+01, greater than the tolerance 1e-12 required for the solution to be considered feasible. Consider increasing the tolerance to be greater than 2.2e+01. If this tolerance is unnaceptably large, the problem is likely infeasible.'
     nit: 1
   slack: array([], dtype=float64)
  status: 2
 success: False
       x: array([0., 0., 0., 0., 0., 0., 0., 0.])

## Тестирование реализованного алгоритма на линейной программе согласно варианту для определения соответствия алгоритма заложенному двойственному симплекс-методу

Рассмотрим ход работы симплекс-метода для линейной программы (заданной в канонической форме) согласно варианту 25:
$$
    \begin{cases}
      c^Tx \rightarrow max \\
      Ax = b \\
      x > 0 \\
     \end{cases}
$$
где: 
$$
    \begin{aligned} 
        A &= \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
    -1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
    1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
    1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
    \end{pmatrix} \\ \\
    b &= \begin{pmatrix} 2 & 10 & 25 & -4\end{pmatrix} \\ \\
    c &= \begin{pmatrix} 12 & -24 & -6 & -34 & -1 & 4 & 7 & 3 & 23  \end{pmatrix} 
    \end{aligned}
$$
при этом необходимо взять начальный двойственный план $y^0$:
$$
    y^0 = \begin{pmatrix} 4 & -5 & 1 & 3\end{pmatrix}
$$
и базис:
$$
    J^0_B = \{2, 6, 7, 8\}
$$

**Итерация** $i = 0$ 

По заданному базису построим $A_B$ и $B = (A_B)^{-1}$: $$
\begin{aligned}
A_B &= \begin{pmatrix} 
    -1& -1 & 2 & 1 \\
    2 & 1 & 1 & 1 \\
    -1 & 4 & 4 & 1 \\
    -3 & 3 & 0 & 1
\end{pmatrix} \\
B &= \begin{pmatrix}
  -0.125 & 0.25 & 0 & -0.125\\
  -0.25 & 0.056 & 0.111 & 0.083\\
  0.125 & -0.139 & 0.222 & -0.208\\
  0.375 & 0.583 & -0.333 & 0.375\\
\end{pmatrix}
\end{aligned}
$$

Вычислим соответствующий базисному двойственному плану коплан:
$$
    \delta^0 = {y^{0}}^TA - c^T = \begin{pmatrix} 4 & -5 & 1 & 3 \end{pmatrix} \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
    -1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
    1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
    1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
    \end{pmatrix} - \begin{pmatrix} 12 & -24 & -6 & -34 & -1 & 4 & 7 & 3 & 23  \end{pmatrix} = \begin{pmatrix} 1 & 0 & 2 & 3 & 4 & 0 & 0 & 0 & 5 \end{pmatrix}
$$

Вычислим базисные компоненты псевдоплана $\chi_B$:
$$\chi_B = Bb = \begin{pmatrix}
  -0.125 & 0.25 & 0 & -0.125\\
  -0.25 & 0.056 & 0.111 & 0.083\\
  0.125 & -0.139 & 0.222 & -0.208\\
  0.375 & 0.583 & -0.333 & 0.375\\
\end{pmatrix} \begin{pmatrix} 2 & 10 & 25 & -4\end{pmatrix} = \begin{pmatrix} 2.75 & 2.5 & 5.25 & -3.25 \end{pmatrix}$$

Условие $\chi_B \ge 0$ не выполняется, поэтому продолжаем итерацию. Находим базисный индекс $j_k \in J^0_B$ для которого $X_{j_k} < 0$:
$$k = 4, \quad j_k = j_2 = 8$$

Находим числа:
$$\mu_j = B^T_kA_j, \quad \forall j \in J$$
где $B_k^T$ - $k$ строка матрицы $B$, $A_j$ - $j$-ый столбец матрицы $A$. Таким образом:
$$\mu = \begin{pmatrix} -0.167 & 0 & 0.25 & -0.5 & 0.042 & 0 & 0 & 1. & 2.917\end{pmatrix}$$

Вычисляем шаги $\sigma_j, \quad \forall j \in J^0_H = \{1, 3, 4, 5\}$ и $\sigma_{j_s}$ по формулам:
$$\sigma_j = \begin{cases}
-\frac{\sigma_j}{\mu_j}, \quad \text{if } \mu_j < 0 \\
+\infty, \quad \text{if } \mu_j \ge 0
\end{cases} j \in J^0_H\\ 
\sigma_{j_s} = \min_{j \in J^0_H}\sigma_j$$

Получим:
$$\sigma_1 = 6, \quad \sigma_3 = +\infty, \quad \sigma_4 = 6, \quad \sigma_5 = +\infty\\
\sigma_{j_s} = \sigma_{j_3} = \sigma_{4} = 6, \quad s = 3, j_s = 4$$
Построим новый двойственный план: 
$$y^1 = y^0 + \sigma_{j_s}e_k^TB =  \begin{pmatrix} 4 & -5 & 1 & 3\end{pmatrix} + 6 * \begin{pmatrix}0.375 & 0.583 & -0.333 & 0.375\end{pmatrix} = \begin{pmatrix} 6.25 & -1.5 & -1 & 5.25 \end{pmatrix}$$
соответствующий ему новый коплан:
$$\delta^1 = y^1A - c =  \begin{pmatrix} 6.25 & -1.5 & -1 & 5.25 \end{pmatrix} \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
    -1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
    1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
    1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
    \end{pmatrix} - \begin{pmatrix} 12 & -24 & -6 & -34 & -1 & 4 & 7 & 3 & 23  \end{pmatrix} = \begin{pmatrix} 0 & 0 & 3.5 & 0 & 4.25 & 0 & 0 & 6 & 22.5\end{pmatrix}$$
новый базис $J^1_B = \{2, 6, 7, 4\}$, новую матрицу $A_B$:
$$A_B = \begin{pmatrix}
  -1. & -1. & 2. & 2.\\
  2. & 1. & 1. & 3.\\
  -1. & 4. & 4. & 0.\\
  -3. & 3. & 0. & -8.\\
\end{pmatrix}$$
и новую матрицу $B$:
$$B = B - \frac{B(A_{j_s} - A_{j_k}e^T_kB)}{e^T_kBA_{j_s}}= \begin{pmatrix}
  1. & 2. & -1. & 1.\\
  -1. & -1.111 & 0.778 & -0.667\\
  1.25 & 1.611 & -0.778 & 0.917\\
  -0.75 & -1.167 & 0.667 & -0.75\\
\end{pmatrix}$$

**Итерация** $i = 1$

Вычислим базисные компоненты псевдоплана $\chi_B$:
$$\chi_B = Bb = \begin{pmatrix}
  1. & 2. & -1. & 1.\\
  -1. & -1.111 & 0.778 & -0.667\\
  1.25 & 1.611 & -0.778 & 0.917\\
  -0.75 & -1.167 & 0.667 & -0.75\\
\end{pmatrix} \begin{pmatrix} 2 & 10 & 25 & -4\end{pmatrix} = \begin{pmatrix}-7. & 9. & -4.5 & 6.5\end{pmatrix}$$
Условие $\chi_B \ge 0$ не выполняется, продолжаем итерацию. Находим базисный индекс $j_k \in J^1_B$, для которого $\chi_{j_k} < 0$:
$$k = 1, j_k = j_1 = 2$$
Находим числа:
$$\mu_j = B^T_kA_j, \quad \forall j \in J \\
\mu = \begin{pmatrix}-1. & 1. & 1. & -0. & 0. & 0. & 0. & 3. & 8.\end{pmatrix}$$
Вычисляем шаги $\sigma_j, \quad \forall j \in J^1_H = \{1, 3, 8, 5\}$ и $\sigma_0$:
$$\sigma_1 = 0, \quad \sigma_3 = +\infty, \quad \sigma_8 = +\infty, , \quad \sigma_5 = +\infty \\
\sigma_{j_s} = \sigma_{j_1} = \sigma_1 = 0, \quad s = 1, j_1 = 1$$

Таким образом, полученный новый двойственный базисный план $y_2$ будет совпадать с текущим базисным планом $y_1$, аналогично новый коплан $\delta_2$ будет совпадать с текущим копланом $\delta_1$. Изменится базис $J^2_B = \{1, 6, 7, 4\}$ и соответствующие базису матрицы $A_B$:
$$A_B = \begin{pmatrix}
  1. & -1. & 2. & 2.\\
  -1. & 1. & 1. & 3.\\
  1. & 4. & 4. & 0.\\
  1. & 3. & 0. & -8.\\
\end{pmatrix}$$
и $B$:
$$B = \begin{pmatrix}
  -1. & -2. & 1. & -1.\\
  -0.778 & -0.667 & 0.556 & -0.444\\
  1.028 & 1.167 & -0.556 & 0.694\\
  -0.417 & -0.5 & 0.333 & -0.417\\
\end{pmatrix}$$
Аналогично проводятся последующие итерации.

Запустив реализованный алгоритм можно увидеть, что выводимые промежуточные результаты совпадают c разобранным шагами двойственного симплекс-метода. 

In [25]:
A = np.array(
    [
        [1, -1, 0, 2, -1, -1, 2, 1, 7],
        [-1, 2, 1, 3, 0, 1, 1, 1, 1],
        [1, -1, 1, 0, 1, 4, 4, 1, 2],
        [1, -3, 0, -8, 2, 3, 0, 1, 1],
    ],
    dtype="float32",
)

b = np.array([2, 10, 25, -4], dtype="float32")
c = np.array([12, -24, -6, -34, -1, 4, 7, 3, 23], dtype="float32")

initial_solution = np.array([4, -5, 1, 3], dtype="float32")
initial_basis = np.array([1, 5, 6, 7], dtype="int")
with np.printoptions(precision=3, suppress=True):
    get_solution(
        *dual_simplex_method(A, b, c, initial_solution, initial_basis, verbose=True), c.size
    )

Iteration no.: 0
Solution: [ 4. -5.  1.  3.]
Basis: [2 6 7 8]
Cosolution: [1. 0. 2. 3. 4. 0. 0. 0. 5.]
A_B: 
[[-1. -1.  2.  1.]
 [ 2.  1.  1.  1.]
 [-1.  4.  4.  1.]
 [-3.  3.  0.  1.]]
Inversed A_B: 
[[-0.125  0.25   0.    -0.125]
 [-0.25   0.056  0.111  0.083]
 [ 0.125 -0.139  0.222 -0.208]
 [ 0.375  0.583 -0.333  0.375]]
x_B: [ 2.75  2.5   5.25 -3.25]
k: 4
j_k: 8
mu_j: [-0.167 -0.     0.25  -0.5    0.042  0.     0.     1.     2.917]
Sigma: [6. 6.]
s: 3
j_s: 4
Iteration no.: 1
Solution: [ 6.25 -1.5  -1.    5.25]
Basis: [2 6 7 4]
Cosolution: [ 0.    0.    3.5   0.    4.25  0.    0.    6.   22.5 ]
A_B: 
[[-1. -1.  2.  2.]
 [ 2.  1.  1.  3.]
 [-1.  4.  4.  0.]
 [-3.  3.  0. -8.]]
Inversed A_B: 
[[ 1.     2.    -1.     1.   ]
 [-1.    -1.111  0.778 -0.667]
 [ 1.25   1.611 -0.778  0.917]
 [-0.75  -1.167  0.667 -0.75 ]]
x_B: [-7.   9.  -4.5  6.5]
k: 1
j_k: 2
mu_j: [-1.  1.  1. -0.  0.  0.  0.  3.  8.]
Sigma: [0.]
s: 1
j_s: 1
Iteration no.: 2
Solution: [ 6.25 -1.5  -1.    5.25]
Basis: [1 6 